#  Camada Bronze – Ingestão de Dados

##  Documentação

**Objetivo:**  
Realizar a ingestão bruta dos arquivos `yellow_tripdata_2025-MM.parquet` diretamente de uma URL pública, com lógica de retry e salvamento no volume bronze.

**Entradas:**  
- URL pública com os arquivos de viagens mensais em formato Parquet.

**Saídas:**  
- Arquivos salvos na camada `bronze` (`/Volumes/lakehouse/bronze/filestore/`).

**Transformações aplicadas:**  
- Nenhuma transformação nos dados (cópia bruta).
- Retry automático caso falhe por erro HTTP.
- Skip do mês caso o arquivo não seja encontrado (erro 403).

##  Decisões Tomadas

- Adotado o volume Delta para garantir estruturação futura.
- Retry configurado com 5 tentativas e 5s de espera.
- Controle de erro simples para permitir automação no pipeline.

In [0]:
import requests
import datetime
import pandas
import pyspark.sql.functions as F
import time


In [0]:
months_to_ingest = dbutils.widgets.get("months_to_ingest") 
year_to_ingest = dbutils.widgets.get("year_to_ingest")

if not( int(months_to_ingest) <= 12) == True:
    months_to_ingest = 1

In [0]:
url_base = "https://d37ci6vzurychx.cloudfront.net/trip-data/"
bronze_path = f"/Volumes/lakehouse/bronze/filestore/{year_to_ingest}/"

max_retries = 5
retry_delay = 5  # seconds

for month in range(1, months_to_ingest + 1):
    try:
        month_str = f"{month:02d}"
        file_date = f"yellow_tripdata_{year_to_ingest}-{month_str}.parquet"
        url = url_base + file_date
        retries = 0

        while retries < max_retries:
            try:
                dbutils.fs.cp(url, bronze_path)  # try to copy the file
                break
            except Exception as e:
                if "403" in str(e):  # file not found
                    print(f"File not found: {file_date}")
                    break
                retries += 1
                if retries >= max_retries:
                    raise e
                time.sleep(retry_delay)  # wait before retrying

    except Exception as e:
        print(f"Error copying {file_date}: {e}")

In [0]:
df = spark.read.parquet(bronze_path)
df= df.withColumn("date_processing", F.current_timestamp ())

df.write.mode("append").saveAsTable("lakehouse.bronze.yellow_trip")
